In [21]:
#!pip install --upgrade ferpy

This notebook shows how to report an experimental result using `ferpy`.
The example shown here is taken from JCGM 100:2008, Example H.2: Simultaneous resistance and reactance measurement.

That example states the following problem:

> The resistance $R$ and the reactance $X$ of a circuit element are determined by measuring the amplitude $V$ of a
sinusoidally-alternating potential difference across its terminals, the amplitude $I$ of the alternating current
passing through it, and the phase-shift angle
$\Phi$ of the alternating potential difference relative to the alternating
current. Thus the three input quantities are V, I, and
$Φ$ and the three output quantities — the measurands —
are the three impedance components $R$, $X$, and $Z$. Since $Z^2 = R^2 + X^2$, there are only two independent output
quantities.

>The measurands are related to the input quantities by Ohm's law:

>\begin{equation}
R = \frac{V}{I} \cos \Phi ; \quad X = \frac{V}{I} \sin \Phi ; \quad Z = \frac{V}{I} ;
\end{equation}

The example says that we make five independent sets of simultaneous observations of the three input quantities $V$, $I$, and $\Phi$. We can use `QuantityValues` structures to record those indications. For example:

In [4]:
from ferpy.main.quantity_values import QuantityValues

In [5]:
qv1 = QuantityValues(
    name="First observation",
    description="Simultaneous observation of $V$, $I$ and $\Phi$",
    quantities=["Potential difference", "Current intensity", "Phase-shift angle"],
    symbols=["V", "I", "\Phi"],
    units=["\Omega","mA","rad"],
    values=[[5.007],[19.663],[1.045_6]],
)

Of course, these Python objects can be used as templates, and filled automatically:

In [6]:
#These are the simultaneous indications
V_indications = [5.007, 4.994, 5.005, 4.990, 4.999]
I_indications = [19.663, 19.639, 19.640, 19.685, 19.678]
Phi_indications = [1.045_6, 1.043_8, 1.046_8, 1.042_8, 1.043_3]

names = ["First observation", "Second observation", "Third observation", "Fourth observation", "Fifth observation"]

all_qv = []

for i in range(len(V_indications)):
  temp_qv = QuantityValues(
      name=names[i],
      description="Simultaneous observation of $V$, $I$ and $\Phi$",
      quantities=["Potential difference", "Current intensity", "Phase-shift angle"],
      symbols=["V", "I", "\Phi"],
      units=["\Omega","mA","rad"],
      values=[V_indications[i],I_indications[i],Phi_indications[i]],
      correlation_indices=[0,1,2]
  )

  all_qv.append(temp_qv)

Now we can have 5 `Measurement` objects to represent those 5 sets of independent measurements: the quantity values defined above will be the results of those measurements, and their sources will be a description of the instrument used to measure them. As they are raw indications, the `input_quantities` of the `Source` will be `null`.

In [7]:
from ferpy.main.measurement import Measurement
from ferpy.main.source import Source

In [8]:
all_indication_measurements = []

for qv in all_qv:

  source = Source(
      name="Instrument to measure $V$, $I$ and $\Phi$",
      description="""
        The instrument measures the amplitude $V$ of a
        sinusoidally-alternating potential difference across its terminals,
        the amplitude $I$ of the alternating current
        passing through it, and the phase-shift angle $\Phi$ of the alternating
        potential difference relative to the alternating
        current.
      """,
      correlations=None,
      input_quantities=[]
  )

  measurement = Measurement(
      description="Simultaneous measurement of $V$, $I$ and $\Phi$",
      correct = True,
      results = [
          qv
      ],
      correlations = None,
      measurands = ["Potential difference", "Current intensity", "Phase-shift angle"],
      source = source
  )

  all_indication_measurements.append(measurement)

We can see the structure of one of the resulting `Measurement` objects:

In [9]:
all_indication_measurements[0].to_dict()

{'id': 'd7cffb62-131e-412c-9670-dfa1ae5d9021',
 'description': 'Simultaneous measurement of $V$, $I$ and $\\Phi$',
 'changelog': None,
 'correct': True,
 'state': None,
 'results': [{'id': '8c1187d3-2062-4d5c-a8dc-73c8071e0bd6',
   'name': 'First observation',
   'description': 'Simultaneous observation of $V$, $I$ and $\\Phi$',
   'changelog': None,
   'quantities': ['Potential difference',
    'Current intensity',
    'Phase-shift angle'],
   'symbols': ['V', 'I', '\\Phi'],
   'units': ['\\Omega', 'mA', 'rad'],
   'values': [5.007, 19.663, 1.0456],
   'standard_uncertainties': None,
   'coverages': None,
   'probability_density_functions': None,
   'correlation_indices': [0, 1, 2]}],
 'correlations': None,
 'measurands': ['Potential difference',
  'Current intensity',
  'Phase-shift angle'],
 'source': {'id': 'b3522418-1825-45eb-871c-7cb68f644428',
  'name': 'Instrument to measure $V$, $I$ and $\\Phi$',
  'description': '\n        The instrument measures the amplitude $V$ of a\n     

In order to report a measuring model, the most suitable way is by extending a `Source` object and implementing the measuring model in its methods:

In [10]:
import numpy as np

In [11]:
class CalculateAverages(Source):

  def compute_averages(self):
    """
    This methods computes the mean and the standard deviation of the input
    quantities and returns a QuantityValues object.
    """
    all_V = []
    all_I = []
    all_Phi = []

    for measurement in self.input_quantities:
      for qv in measurement.results:
        all_V.append(qv.values[0])
        all_I.append(qv.values[1])
        all_Phi.append(qv.values[2])

    # Compute the mean of the independent mesurements
    mean_V = np.mean(all_V)
    mean_I = np.mean(all_I)
    mean_Phi = np.mean(all_Phi)

    #print(all_I)
    #print(all_Phi)

    # Compute the experimental standard deviation
    std_V = np.std(all_V, ddof=1)/np.sqrt(len(all_V))
    std_I = np.std(all_I, ddof=1)/np.sqrt(len(all_I))
    std_Phi = np.std(all_Phi, ddof=1)/np.sqrt(len(all_Phi))

    #print(mean_V, mean_I, mean_Phi)
    #print(std_V, std_I, std_Phi)

    """
    The means are taken as the best estimates of the expected values of the
    input quantities, and the experimental standard deviations are the standard
    uncertainties of those means.
    """

    # Round the standard uncertainties to two significant digits, and then
    # round the means accordingly.

    std_V = self.signif(std_V, 2)
    std_I = self.signif(std_I, 2)
    std_Phi = self.signif(std_Phi, 2)
    #print(std_V, std_I, std_Phi)

    mean_V = self.round_mean(mean_V, std_V)
    mean_I = self.round_mean(mean_I, std_I)
    mean_Phi = self.round_mean(mean_Phi, std_Phi)
    #print(mean_V, mean_I, mean_Phi)

    result_qv = QuantityValues(
        name="Mean values and standard uncertainties of $V$, $I$ and $\Phi$",
        description="Mean values and standard uncertainties of $V$, $I$ and $\Phi$",
        quantities=[
            "Mean potential difference",
            "Mean current intensity",
            "Mean phase-shift angle"
        ],
        symbols=["\bar{V}", "\bar{I}", "\bar{\Phi}"],
        units=["\Omega","mA","rad"],
        values=[[mean_V],[mean_I],[mean_Phi]],
        standard_uncertainties=[[std_V], [std_I], [std_Phi]]
    )

    return result_qv

  def compute_correlation_coefficients(self):

    all_V = []
    all_I = []
    all_Phi = []

    for measurement in self.input_quantities:
      for qv in measurement.results:
        all_V.append(qv.values[0])
        all_I.append(qv.values[1])
        all_Phi.append(qv.values[2])

    return np.corrcoef([all_V, all_I, all_Phi])

  def signif(self, x, p):
    # Taken from:
    # https://stackoverflow.com/questions/18915378/rounding-to-significant-figures-in-numpy
    x = np.asarray(x)
    x_positive = np.where(np.isfinite(x) & (x != 0), np.abs(x), 10**(p-1))
    mags = 10 ** (p - 1 - np.floor(np.log10(x_positive)))
    return np.round(x * mags) / mags

  def round_mean(self, mean, std):

    return np.round(mean, len(str(std).split('.')[-1]))

In [12]:
calculate_average_source = CalculateAverages(
    name="Mean and standard uncertainties of $V$, $I$ and $\Phi$",
    description="Mean and standard uncertainties of $V$, $I$ and $\Phi$",
    input_quantities=all_indication_measurements
)

Once the `Source` object is ready, we can create a `Measurement` that will report the results generated by that source:

In [13]:
mean_measurement = Measurement(
    description="",
    correct=True,
    measurands=[
        "Mean potential difference",
        "Mean current intensity",
        "Mean phase-shift angle"
    ],
    source=calculate_average_source
)

mean_measurement.results = [mean_measurement.source.compute_averages()]

We can also add the correlation matrix to that measurement:

In [14]:
from ferpy.auxiliary.correlations import Correlations

In [15]:
mean_measurement.correlations = Correlations(
    quantities=mean_measurement.measurands,
    correlation_matrix=mean_measurement.source.compute_correlation_coefficients(),
    method="np.corrcoef"
)

Finally we can create the `Source` to compute the measurands of our experiment based in the $\bar{V}$, $\bar{I}$, $\bar{\Phi}$ that we have obtained so far.

In [16]:
class RXZcalculation(Source):

  def update_variables(self):

    results_qv = self.input_quantities[0].results[0]

    self._V = results_qv.values[0][0]
    self._I = results_qv.values[1][0]*1e-3
    self._Phi = results_qv.values[2][0]

    self._u_V = results_qv.standard_uncertainties[0][0]
    self._u_I = results_qv.standard_uncertainties[1][0]*1e-3
    self._u_Phi = results_qv.standard_uncertainties[2][0]

    self._cor_V_I = self.input_quantities[0].correlations.correlation_matrix[0][1]
    self._cor_V_Phi = self.input_quantities[0].correlations.correlation_matrix[0][2]
    self._cor_I_Phi = cor_V_I = self.input_quantities[0].correlations.correlation_matrix[1][2]

  def compute_results(self):
    self.update_variables()

    self._Z = self._V/self._I
    self._R = self._Z*np.cos(self._Phi)
    self._X = self._Z*np.sin(self._Phi)

    #print(self._R, self._X, self._Z)

  def compute_standard_uncertainties(self):

    if None in (self._Z, self._R, self._X):
      self.compute_results()

    self._u_Z = self.Z_standard_uncertainty(
        self._u_V,
        self._V,
        self._u_I,
        self._I,
        self._cor_V_I,
        self._Z
    )

    self._u_R = self.R_standard_uncertainty(
        self._u_V,
        self._V,
        self._u_I,
        self._I,
        self._u_Phi,
        self._Phi,
        self._cor_V_I,
        self._cor_V_Phi,
        self._cor_I_Phi
    )

    self._u_X = self.X_standard_uncertainty(
        self._u_V,
        self._V,
        self._u_I,
        self._I,
        self._u_Phi,
        self._Phi,
        self._cor_V_I,
        self._cor_V_Phi,
        self._cor_I_Phi
    )

    #print(self._u_R, self._u_X, self._u_Z)

    pass

  def Z_standard_uncertainty(self, u_V, V, u_I, I, cor_V_I, Z):
    # compute the relative uncertainties
    u_V_r = u_V/V
    u_I_r = u_I/I

    correlation_factor = -2.0*u_V_r*u_I_r*cor_V_I

    squared_sum = np.sum([u_V_r**2, u_I_r**2, correlation_factor])

    return np.sqrt(squared_sum)*Z

  def R_standard_uncertainty(self, u_V, V, u_I, I, u_Phi, Phi, cor_V_I, cor_V_Phi, cor_I_Phi):

    coef1 = (np.cos(Phi)/I)
    coef2 = (-V*np.cos(Phi)/I**2)
    coef3 = (-V*np.sin(Phi)/I)

    correlation_factor1 = 2.0*coef1*coef2*u_V*u_I*cor_V_I
    correlation_factor2 = 2.0*coef1*coef3*u_V*u_Phi*cor_V_Phi
    correlation_factor3 = 2.0*coef2*coef3*u_I*u_Phi*cor_I_Phi

    squared_sum = np.sum([
        (coef1*u_V)**2,
        (coef2*u_I)**2,
        (coef3*u_Phi)**2,
        correlation_factor1,
        correlation_factor2,
        correlation_factor3,
    ])

    return np.sqrt(squared_sum)

  def X_standard_uncertainty(self, u_V, V, u_I, I, u_Phi, Phi, cor_V_I, cor_V_Phi, cor_I_Phi):

    coef1 = (np.sin(Phi)/I)
    coef2 = (-V*np.sin(Phi)/I**2)
    coef3 = (V*np.cos(Phi)/I)

    correlation_factor1 = 2.0*coef1*coef2*u_V*u_I*cor_V_I
    correlation_factor2 = 2.0*coef1*coef3*u_V*u_Phi*cor_V_Phi
    correlation_factor3 = 2.0*coef2*coef3*u_I*u_Phi*cor_I_Phi

    squared_sum = np.sum([
        (coef1*u_V)**2,
        (coef2*u_I)**2,
        (coef3*u_Phi)**2,
        correlation_factor1,
        correlation_factor2,
        correlation_factor3,
    ])

    return np.sqrt(squared_sum)

  def get_results(self):

    self.compute_results()
    self.compute_standard_uncertainties()

    result_qv = QuantityValues(
      name="Values and standard uncertainties of $R$, $X$ and $Z$",
      description="Values and standard uncertainties of $R$, $X$ and $Z$",
      quantities=[
          "Resistance",
          "Reactance",
          "Impedance"
      ],
      symbols=["R", "X", "Z"],
      units=["\Omega","\Omega","\Omega"],
      values=[[self._R],[self._X],[self._Z]],
      standard_uncertainties=[[self._u_R], [self._u_X], [self._u_Z]]
    )

    return result_qv

In [17]:
RXZ_calculation_source = RXZcalculation()
RXZ_calculation_source.input_quantities = [mean_measurement]

Now we can create the final `Measurement` object to store the results:

In [18]:
final_measurement = Measurement(
    description="",
    correct=True,
    measurands=[
            "Resistance",
            "Reactance",
            "Impedance"
        ],
    source=RXZ_calculation_source
)

final_measurement.results = [final_measurement.source.get_results()]

The final results can be rendered directly using latex:

In [19]:
from IPython.display import display, Math

def render_result_as_latex(results, i):
    """
    Renders a single result as a LaTeX string for displaying in Jupyter notebooks.

    :param results: Object containing quantities, symbols, values, and uncertainties.
    :param i: Index of the quantity to display.
    """
    # Format the value and uncertainty to two decimal places
    value = f"{results.values[i][0]:.2f}"
    uncertainty = f"{results.standard_uncertainties[i][0]:.2f}"

    # Build the LaTeX string
    latex_str = (
        r"\mathrm{" + results.quantities[i] + "}: "  # Quantity name
        + results.symbols[i]  # Symbol
        + r" = (" + value + r" \pm " + uncertainty + r") \, \Omega"  # Value and uncertainty
    )

    # Render LaTeX in the notebook
    display(Math(latex_str))

In [20]:
results = final_measurement.results[0]

for i in range(len(results.values)):
  render_result_as_latex(results, i)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>